In [ ]:
# 필요한 utils
import pandas as pd
import numpy as np

# import sys
# sys.path.append("../3.src")
# from utils import *

# !cat ../3.src/utils.py


# 3. csv 파일 불러오기
DATA_PATH = "/data/ephemeral/home/workspace/upstageailab-ml-competition-ml-2/1.Data/"
train_df = pd.read_csv(DATA_PATH + "train.csv")
test_df = pd.read_csv(DATA_PATH + "test.csv")

subway_df = pd.read_csv(DATA_PATH + "subway_feature.csv")
bus_df = pd.read_csv(DATA_PATH + "bus_feature.csv")

In [48]:
# train 데이터와 test 데이터 하나로 합치기! 
train_df['is_train'] = 1
test_df['is_train'] = 0
all_df = pd.concat([train_df, test_df], ignore_index= True)

In [50]:
all_df = all_df.rename(columns={'전용면적(㎡)': '전용면적'})

In [ ]:
# 파생 변수 1. building_age
all_df['building_age'] = all_df['건축년도'].apply(lambda x : 2025 - x if x != 0 else 0)

In [ ]:
# 파생 변수 2. years_diff_since_contract
all_df['years_diff_since_contract'] = np.where(
    (all_df['계약년월'].notnull()) & (all_df['계약년월'] != 0)
    ,2025 - all_df['계약년월'].astype(str).str[:4].astype(int)
    ,0
)

In [56]:
# 파생 변수 3. parking_per_household
all_df['parking_per_household'] = np.where(
    (all_df['주차대수'].notnull()) & (all_df['k-전체세대수'].notnull()) & (all_df['k-전체세대수'] != 0),
    all_df['주차대수'] / all_df['k-전체세대수'],
    0        
) 

In [ ]:
# 파생 변수 4. nearnest_distance_to_bus  &  num_bus_stops_within_1000m

from sklearn.neighbors import BallTree

valid_mask = all_df[['좌표X', '좌표Y']].notnull().all(axis=1)
valid_rows = all_df[valid_mask]

valid_mask = all_df[['좌표X', '좌표Y']].notnull().all(axis=1)
valid_rows = all_df[valid_mask].copy()

bus_radians = np.radians(bus_df[['Y좌표', 'X좌표']].values)
valid_radians = np.radians(valid_rows[['좌표Y', '좌표X']].values)

tree = BallTree(bus_radians, metric='haversine')

# 1)  nearnest_distance_to_bus 
distances, indices = tree.query(valid_radians, k=1)

valid_rows['nearnest_distance_to_bus'] = distances[:, 0] * 6371  

all_df.loc[valid_mask, 'nearnest_distance_to_bus'] = valid_rows['nearnest_distance_to_bus']
all_df.loc[~valid_mask, 'nearnest_distance_to_bus'] = 9999

# 2) num_bus_stops_within_1000m
radius_km = 1.0
radius_rad = radius_km / 6371.0  

indices_array = tree.query_radius(valid_radians, r=radius_rad)

valid_rows['num_bus_stops_within_1000m'] = [len(indices) for indices in indices_array]

all_df['num_bus_stops_within_1000m'] = 0
all_df.loc[valid_mask, 'num_bus_stops_within_1000m'] = valid_rows['num_bus_stops_within_1000m'].values



In [61]:
# 파생 변수 5. nearnest_distance_to_subway   &  num_subway_within_1000m

valid_mask = all_df[['좌표X', '좌표Y']].notnull().all(axis=1)
valid_rows = all_df[valid_mask].copy()

subway_radians = np.radians(subway_df[['위도', '경도']].values)
valid_radians = np.radians(valid_rows[['좌표Y', '좌표X']].values)

tree = BallTree(subway_radians, metric='haversine')

# 1)  nearnest_distance_to_subway 
distances, indices = tree.query(valid_radians, k=1)

valid_rows['nearnest_distance_to_subway'] = distances[:, 0] * 6371  

all_df.loc[valid_mask, 'nearnest_distance_to_subway'] = valid_rows['nearnest_distance_to_subway']
all_df.loc[~valid_mask, 'nearnest_distance_to_subway'] = 9999




# 2) num_subway_within_1000m
radius_km = 1.0
radius_rad = radius_km / 6371.0  

indices_array = tree.query_radius(valid_radians, r=radius_rad)

valid_rows['num_subway_within_1000m'] = [len(indices) for indices in indices_array]

all_df['num_subway_within_1000m'] = 0
all_df.loc[valid_mask, 'num_subway_within_1000m'] = valid_rows['num_bus_stops_within_1000m'].values

In [ ]:
# 파생 변수 6. dist_to_CBD,  dist_to_Gangnam, dist_to_Yeouido

%pip install haversine
from haversine import haversine

center_coords = {
    'CBD':     (37.5714, 126.9768),
    'Gangnam': (37.4979, 127.0276),
    'Yeouido': (37.5219, 126.9245)
}

all_df['dist_to_CBD'] = all_df.apply(lambda row : haversine((row['좌표Y'], row['좌표X']), center_coords['CBD']) if pd.notnull(row['좌표X']) and pd.notnull(row['좌표Y']) else 9999, axis=1)
all_df['dist_to_Gangnam'] = all_df.apply(lambda row : haversine((row['좌표Y'], row['좌표X']), center_coords['Gangnam']) if pd.notnull(row['좌표X']) and pd.notnull(row['좌표Y']) else 9999 , axis=1)
all_df['dist_to_Yeouido'] = all_df.apply(lambda row : haversine((row['좌표Y'], row['좌표X']), center_coords['Yeouido']) if pd.notnull(row['좌표X']) and pd.notnull(row['좌표Y']) else 9999 , axis=1)


In [ ]:
# 파생 변수 1. building_age
# 파생 변수 2. years_diff_since_contract
# 파생 변수 3. parking_per_household
# 파생 변수 4. nearnest_distance_to_bus  &  num_bus_stops_within_1000m
# 파생 변수 5. nearnest_distance_to_subway   &  num_subway_within_1000m
# 파생 변수 6. dist_to_CBD,  dist_to_Gangnam, dist_to_Yeouido

# 파생 변수 7. transport_score(파생 변수 4, 5 를 합치기)
# 거리가 짧을수록 점수가 커지도록...

all_df['inv_nearnest_distance_to_bus'] = 1 / (1 + all_df['nearnest_distance_to_bus'])
all_df['inv_nearnest_distance_to_subway'] = 1 / (1 + all_df['nearnest_distance_to_subway'])

all_df['transport_score'] = (
    all_df['inv_nearnest_distance_to_bus'] +
    all_df['num_bus_stops_within_1000m'] +
    all_df['inv_nearnest_distance_to_subway'] +
    all_df['num_subway_within_1000m']
)

In [66]:
# 파생 변수 8. interaction 변수 추가해보기!! 

# 7-1) 면적 * 위치 = 위치가 좋을 수록 면적당 가격 상승
all_df['area_x_gangnam'] = all_df['전용면적'] * all_df['dist_to_Gangnam']
all_df['area_x_cbd'] = all_df['전용면적'] * all_df['dist_to_CBD']
all_df['area_x_yeouido'] = all_df['전용면적'] * all_df['dist_to_Yeouido']

# 7-2) 나이 * 위치 = 오래된 아파트라도 강남이면 비쌈
all_df['age_x_gangnam'] = all_df['building_age'] * all_df['dist_to_Gangnam']
all_df['age_x_cbd'] = all_df['building_age'] * all_df['dist_to_CBD']

# 7-3) 면적 * 나이 = 큰데 오래된 집과 작은데 새집 등의 가격 차이 
all_df['area_x_age'] = all_df['전용면적'] * all_df['building_age']

# 7-4) 교통 점수 높은데, 강남이면 가격 더 가파르게 상승
all_df['transport_x_gangnam'] = all_df['transport_score'] * all_df['dist_to_Gangnam']

# 7-5) 좌표와 강남까지의 거리의 상호작용
all_df['coordX_x_gangnam'] = all_df['좌표X'] * all_df['dist_to_Gangnam']
all_df['coordY_x_gangnam'] = all_df['좌표Y'] * all_df['dist_to_Gangnam']



In [ ]:
# EDA  1) 결측치 : 80% 이하만 데이터로 활용!  all_df_no_nan
missing_check_mask = all_df.isnull() | all_df.isna() | (all_df == ' ') | (all_df == '-') | (all_df == 'nan')

missing_counts = missing_check_mask.sum()
missing_ratio = missing_counts / len(all_df) 

missing_summary = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing Ratio': missing_ratio
})

columns_to_keep = missing_ratio[missing_ratio < 0.8].index

all_df_no_nan = all_df[columns_to_keep]




In [70]:
# EDA  2) 본번, 부번 float → str
all_df_no_nan['본번'] = all_df_no_nan['본번'].astype(str)
all_df_no_nan['부번'] = all_df_no_nan['부번'].astype(str)

/tmp/ipykernel_566866/2784026796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df_no_nan['본번'] = all_df_no_nan['본번'].astype(str)
/tmp/ipykernel_566866/2784026796.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df_no_nan['부번'] = all_df_no_nan['부번'].astype(str)


In [ ]:
# EDA  3) 결측치 보간 - 범주형은 null 로 , 수치형은 선형보간으로
all_df_no_nan.isnull().sum().sort_values(ascending=False)

continous_cols =[]
categorical_cols =[]

for col in all_df_no_nan.columns:
    if pd.api.types.is_numeric_dtype(all_df_no_nan[col]):
        continous_cols.append(col)
    else:
        categorical_cols.append(col)

# 범주형은 null
all_df_no_nan[categorical_cols] = all_df_no_nan[categorical_cols].fillna('NULL')

# 수치형은 선형 보간
all_df_no_nan[continous_cols] = all_df_no_nan[continous_cols].interpolate(method='linear', axis=0)        
            


In [ ]:
# EDA  4) 이상치 제거 - IQR (all_df_no_nan 에서 is_train ==1 인 데이터에서만 이상치 행을 제거!!)

def remove_outliers_iqr(df, col_name):
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]

# is_train ==1 인 데이터에서만   전용면적에 대한 이상치 제거
final_df = pd.concat([
    all_df_no_nan[all_df_no_nan['is_train']==0] 
    ,remove_outliers_iqr(all_df_no_nan[all_df_no_nan['is_train']== 1], '전용면적')
    ])



In [ ]:
# EDA  5) 범주형 변수 처리 - label encoding
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# 나중에 후처리/복원/변환할 때 재사용하기 위해 컬럼 별로 학습된 label endocer 객체를 딕셔너리에 저장!
label_encoders ={}

final_train = all_df_no_nan[all_df_no_nan['is_train'] == 1].copy()
final_test = all_df_no_nan[all_df_no_nan['is_train'] == 0].copy()

for col in tqdm(categorical_cols):
    le = LabelEncoder()

    le.fit(final_train[col].astype(str))
    final_train[col] =le.transform(final_train[col].astype(str))
    label_encoders[col] = le

    for label in final_test[col].unique():
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    final_test[col] = le.transform(final_test[col].astype(str))      


final_df = pd.concat([final_train, final_test], ignore_index=True)